In [1]:
import numpy as np
from scipy.stats import norm

import ipywidgets as w
import bqplot as bq
import bqplot.pyplot as plt
from bqplot import *

In [2]:
def squared_exponential(x1, x2, sigma=1., l=1.):
    z = (x1 - x2[:, np.newaxis]) / l
    return sigma**2 * np.exp(-.5 * z ** 2)

In [3]:
def gp_regression(x_train, y_train, x_test,
                  kernel=squared_exponential,
                  sigma_noise=.1,
                  params=dict(sigma=1., l=1.)):
    # compute the kernel matrices for train, train_test, test combinations
    K = kernel(x_train, x_train, **params)
    K_s = kernel(x_train, x_test, **params)
    
    
    
    
    K_ss = kernel(x_test, x_test, **params)
    
    n, p = len(x_train), len(x_test)
    
    # compute the posterior mean and cov
    mu_s = np.dot(K_s, np.linalg.solve(K + sigma_noise**2 * np.eye(n), y_train))
    cov_s = K_ss - np.dot(K_s, np.linalg.solve(K + sigma_noise**2 * np.eye(n), K_s.T))
    
    # prior and posterior means and cov matrices
    mu_prior, cov_prior = np.zeros(p), K_ss
    mu_post, cov_post = mu_s, cov_s + sigma_noise**2
    
    return dict(prior=(mu_prior, cov_prior), 
                posterior=(mu_post, cov_post))

In [5]:
def expected_improvement(x_test, x_train, y_train, xi=0.01):
    # run gp regession for x_test
    gp_res = gp_regression(x_train, y_train, x_test,
                           sigma_noise=.1,
                           params=dict(sigma=1., l=1.))

    mu_prior, cov_prior = gp_res['prior']
    mu_post, cov_post = gp_res['posterior']

    mu = mu_prior
    sigma = np.sqrt(np.diag(cov_prior))
    
    # compute y_pred by interpolating mu_post at x_train points
    y_pred = np.interp(x_train, x_test, mu_post)
    mu_plus = np.max(y_pred)

    Z = (mu - mu_plus - xi) / sigma
    ei = (mu - mu_plus - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
    ei[sigma == 0.0] = 0.0

    return ei

In [6]:
# ground truth
def f(x): return -np.exp(-x) * np.sin(5 * x)
x = np.linspace(0.5, 2.5, 500)
y = f(x)

In [8]:
gp_fig = plt.figure(title='Bayesian Optimization', 
                    animation_duration=750, 
                    layout=dict(width='1000px', height='600px'))
plt.scales(scales={'y': bq.LinearScale(min=-.6, max=.6)})
plt.plot(x, y, colors=['limegreen'], labels=['Ground Truth'], display_legend=True)
train_scat = plt.scatter([], [], colors=['magenta'], 
                         enable_move=True,
                         labels=['Function Evaluations'],
                         display_legend=True,
                         interactions={'click': 'add'},
                         marker_size=1, marker='square')
gp_line = plt.plot(x, [], 'm')
std_bands = plt.plot(x, [],
                     fill='between',
                     fill_colors=['yellow'],
                     fill_opacities=[.2], stroke_width=0)
plt.xlabel('X')
plt.ylabel('Y')

std_bands_cb = w.Checkbox(description='Display Std Bands?')

# acquisition function
xi_slider = w.FloatSlider(description='$\\xi$', value=.01, min=0, max=.1, step=.01)
af_fig = plt.figure(title='Acquisition Function', 
                    layout=dict(width='1000px', height='300px'))
acq_func_line = plt.plot(x, [], colors=['salmon'], 
                         label='Acquisition Function',
                         display_legend=True)

def update_reg_line(*args):
    global gp_res, x_train, y_train
    x_train = train_scat.x
    y_train = train_scat.y
    
    gp_res = gp_regression(x_train, y_train, x, sigma_noise=0.01)
    mu_post, cov_post = gp_res['posterior']
    
    # update the regression line to the mean of the posterior distribution
    gp_line.y = mu_post

    sig_post = np.sqrt(np.diag(cov_post))
    # update the std bands to +/- 2 sigmas from the posterior mean
    std_bands.y = [mu_post - 2 * sig_post, mu_post + 2 * sig_post]

    update_acq_func(None)
    

def update_acq_func(*args):
    mu_post, cov_post = gp_res['posterior']

    mu = mu_post
    sigma = np.sqrt(np.diag(cov_post))

    # compute y_pred by interpolating mu_post at x_train points
    y_pred = np.interp(x_train, x, mu_post)
    mu_plus = np.max(y_pred)

    xi = xi_slider.value
    Z = (mu - mu_plus - xi) / sigma
    exp_improvement = (mu - mu_plus - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
    exp_improvement[sigma == 0.0] = 0.0

    acq_func_line.y = exp_improvement
    
_ = w.jslink((std_bands_cb, 'value'), (std_bands, 'visible'))
train_scat.observe(update_reg_line, 'x')
xi_slider.observe(update_acq_func)

w.VBox([w.HBox([gp_fig, std_bands_cb]), w.HBox([af_fig, xi_slider])])